<a href="https://colab.research.google.com/github/songyang1995/hello_world/blob/master/%E6%9B%BF%E4%BB%A3%E8%AE%A1%E9%87%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %%
import re
from typing import List
import pandas as pd
import numpy as np
import os, sys


def convert_source_bib_to_list(path: str = r"F:\workspace\pycharm\diagnosis_of_ameloblastoma\bib\savedrecs (1).txt") \
        -> List[dict]:
    """
    将一份WoS导出的 txt文件转换为list[dict]形式的数据
    :param path:
    :return: List[dict]
    """
    res = []
    with open(path, "r", encoding="utf8") as src_file:
        bib_dict = {}
        current_key = ''
        # pp = 0
        for line in src_file:
            # if pp<6:
            #     print(line)
            #     pp+=1
            if len(line) == 0:
                pass
            elif line.startswith("PT"):
                bib_dict = {"PT": line.split()[1]}
                current_key = 'PT'
            elif line.startswith("  "):
                bib_dict[current_key] += "\n" + line.strip()
            elif line.startswith("ER"):
                res.append(bib_dict)
            elif re.match(r"[A-Z]{2}\s.*", line):
                k, v = line.split(maxsplit=1)
                bib_dict[k] = v.strip()
                current_key = k
    return res


def convert_folder_bib_to_list(folder_path: str = r"drive/MyDrive/引文文件夹") -> \
        (List[dict], set):
    """
    获取文件夹中所有文档的pandas格式数据
    :param folder_path: 文件夹位置
    :return:list[dict],set list形式引文信息，标签集合
    """
    article_list_merge = []
    fields = set()
    for file_name in os.listdir(folder_path):
        if not file_name.endswith("txt"):
            continue
        else:
            article_dict = convert_source_bib_to_list(folder_path + "/" + file_name)
            for article_meta in article_dict:
                article_meta["source_file"] = file_name
                fields.update(article_meta.keys())
                article_list_merge.append(article_meta)
            print(f"file {file_name} has processed")
    return article_list_merge, fields


def get_pandas_dataframe_from_bib_list(article_list: List[dict], fields: set):
    """
    转化为pandas格式数据
    """
    res = pd.DataFrame.from_records(article_list)
    return res


In [ ]:
# 批处理
bibs, fields = convert_folder_bib_to_list()
pd_bibs = get_pandas_dataframe_from_bib_list(bibs, fields)

#过滤掉所有不包含DOI字段的数据
pd_bibs = pd_bibs.drop(pd_bibs[(pd_bibs["DI"].isna()) | (pd_bibs["DI"]=="")].index)

# 获取所有DOI csv格式的
pd_bibs[["source_file","DI"]].to_csv("doi列表.csv")
pd_bibs[["source_file","DI"]]

file savedrecs (1).txt has processed


,source_file,DI
0,savedrecs (1).txt,10.1016/j.ijinfomgt.2019.02.009
1,savedrecs (1).txt,10.1016/j.ijinfomgt.2019.01.003
2,savedrecs (1).txt,10.1016/j.ijinfomgt.2019.03.003
3,savedrecs (1).txt,10.1016/j.ijinfomgt.2019.02.006
4,savedrecs (1).txt,10.1016/j.ijinfomgt.2019.03.005
...,...,...
495,savedrecs (1).txt,10.1016/j.ipm.2019.102070
496,savedrecs (1).txt,10.1016/j.ipm.2019.102061
497,savedrecs (1).txt,10.1016/j.ipm.2019.102075
498,savedrecs (1).txt,10.1016/j.ipm.2019.102079


In [ ]:
# 挂载谷歌硬盘，存放数据的地方
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
